In [1]:
import numpy as np 

data_path = "/data/qgong/data/AE/CICDDoS2019/Data/cicddos"

X_train = np.load(data_path+"/X_train.npy")
y_train = np.load(data_path+"/y_train.npy")
X_test = np.load(data_path+"/X_test.npy")
y_test = np.load(data_path+"/y_test.npy")

X_adv = np.load(data_path+"/Adv/Adv_X_test.npy")
y_adv = np.copy(y_test)

print(np.unique(y_train))
print(np.unique(y_test))
print(np.unique(y_adv))

X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_adv.shape, y_adv.shape

[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10]
[ 0  1  2  3  4  5  6  7  8  9 10]


((490783, 78), (490783, 78), (490783,), (490783,), (490783, 78), (490783,))

In [2]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-12-04 13:57:41.648549: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-04 13:57:41.648583: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-04 13:57:41.648608: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 13:57:41.654501: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available:  4


2024-12-04 13:57:42.815563: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-04 13:57:42.815828: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-04 13:57:42.816067: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score


class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_val, y_val):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val

    def on_epoch_end(self, epoch, logs=None):
        # Get predictions
        y_pred = np.argmax(self.model.predict(self.X_val, verbose=0), axis=-1)

        # Calculate F1 score
        f1 = f1_score(self.y_val, y_pred, average='weighted')

        # Print metrics with six significant digits
        print(f"Epoch {epoch + 1:03d}: "
              f"loss={logs['loss']:.6f}, "
              f"accuracy={logs['accuracy']:.6f}, "
              f"val_loss={logs['val_loss']:.6f}, "
              f"val_accuracy={logs['val_accuracy']:.6f}, "
              f"F1={f1:.6f}")



# Define a more practical classifier architecture
def create_classifier(input_dim, output_dim):
    inputs = Input(shape=(input_dim,))
    x = Dense(128, activation='relu')(inputs)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(32, activation='relu')(x)
    outputs = Dense(output_dim, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Parameters
input_dim = 78  # Replace with the number of features in your tabular dataset
output_dim = 11  # Replace with the number of classes

# Create the classifier
classifier = create_classifier(input_dim, output_dim)

# Compile the classifier
classifier.compile(optimizer=Adam(learning_rate=0.001),
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])


# Pre-train the classifier
# classifier.fit( X_train, 
#                 y_train, 
#                 epochs=50, 
#                 batch_size=32, 
#                 validation_data=(X_test, y_test),
#                 callbacks=[MetricsCallback(X_test, y_test)])

classifier.fit( X_train, 
                y_train, 
                epochs=20, 
                batch_size=64, 
                validation_split=0.2)

# Save the weights
classifier.save_weights('classifier.weights.h5')

2024-12-04 13:57:43.450293: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-04 13:57:43.450561: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-04 13:57:43.450797: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/20


2024-12-04 13:57:44.906030: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x652c9479e9a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-12-04 13:57:44.906058: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX 6000 Ada Generation, Compute Capability 8.9
2024-12-04 13:57:44.906064: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA RTX 6000 Ada Generation, Compute Capability 8.9
2024-12-04 13:57:44.906068: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (2): NVIDIA RTX 6000 Ada Generation, Compute Capability 8.9
2024-12-04 13:57:44.906072: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (3): NVIDIA RTX 6000 Ada Generation, Compute Capability 8.9
2024-12-04 13:57:44.910451: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODU

6135/6135 [==============================] - 57s 9ms/step - loss: 0.4780 - accuracy: 0.8218 - val_loss: 0.3339 - val_accuracy: 0.8701
Epoch 2/20
6135/6135 [==============================] - 55s 9ms/step - loss: 0.3554 - accuracy: 0.8606 - val_loss: 0.3105 - val_accuracy: 0.8752
Epoch 3/20
6135/6135 [==============================] - 55s 9ms/step - loss: 0.3342 - accuracy: 0.8676 - val_loss: 0.3094 - val_accuracy: 0.8791
Epoch 4/20
6135/6135 [==============================] - 55s 9ms/step - loss: 0.3238 - accuracy: 0.8717 - val_loss: 0.2916 - val_accuracy: 0.8851
Epoch 5/20
6135/6135 [==============================] - 55s 9ms/step - loss: 0.3167 - accuracy: 0.8730 - val_loss: 0.2959 - val_accuracy: 0.8776
Epoch 6/20
6135/6135 [==============================] - 55s 9ms/step - loss: 0.3123 - accuracy: 0.8752 - val_loss: 0.2897 - val_accuracy: 0.8815
Epoch 7/20
6135/6135 [==============================] - 54s 9ms/step - loss: 0.3077 - accuracy: 0.8763 - val_loss: 0.2811 - val_accuracy: 0.8

In [4]:
from sklearn.metrics import precision_score, recall_score, f1_score

pretrained_classifier = create_classifier(input_dim, output_dim)
pretrained_classifier.load_weights('classifier.weights.h5')
# pretrained_classifier.compile()

pretrained_classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Verify the loaded model works
loss, accuracy = pretrained_classifier.evaluate(X_test, y_test, verbose=0)

# Predict class probabilities and get the predicted classes
y_pred_prob = pretrained_classifier.predict(X_test)
y_pred = y_pred_prob.argmax(axis=1)

# Compute precision, recall, and F1 score using scikit-learn
precision = precision_score(y_test, y_pred, average='weighted')  # Weighted for multi-class
recall = recall_score(y_test, y_pred, average='weighted')        # Weighted for multi-class
f1 = f1_score(y_test, y_pred, average='weighted')                # Weighted for multi-class

# Print the metrics
print(f"Accuracy: {accuracy:.6f}")
print(f"Precision: {precision:.6f}")
print(f"Recall: {recall:.6f}")
print(f"F1 Score: {f1:.6f}")

15337/15337 [==============================] - 65s 4ms/step
Accuracy: 0.889413
Precision: 0.900074
Recall: 0.889413
F1 Score: 0.890361


In [5]:
# from alibi_detect.ad import AdversarialAE
# from alibi_detect.utils.saving import save_detector

# # Define encoder and decoder (same as before or custom)
# encoder_net = tf.keras.Sequential(
#     [
#         Input(shape=(input_dim,)),
#         Dense(64, activation='relu'),
#         Dense(32)  # Latent representation
#     ]
# )

# decoder_net = tf.keras.Sequential(
#     [
#         Input(shape=(32,)),
#         Dense(64, activation='relu'),
#         Dense(input_dim)  # Reconstruct original input
#     ]
# )

# # Initialize AdvAE with the pre-trained classifier
# ad = AdversarialAE(
#     encoder_net=encoder_net,
#     decoder_net=decoder_net,
#     model=pretrained_classifier
# )

# # Train the AdvAE
# ad.fit(X_train, epochs=40, batch_size=64, verbose=True)

# # Save the trained AdvAE detector
# save_detector(ad, 'adv_ae_detector_with_pretrained_classifier')
